In [2]:
import scanpy as sc
import anndata

In [16]:
ann = sc.read('../merge/4.ann.h5ad')
ann.obs_names = ann.obs_names.map(lambda x: x[:-3] if x.endswith('dv-dv') or x.endswith('bl-bl') else x)
ann

AnnData object with n_obs × n_vars = 40350 × 4000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'mt_outlier', 'genes_outlier', 'batch', 'leiden_res', 'cluster', 'cluster_name'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
    uns: 'batch_colors', 'hvg', 'leiden_res', 'leiden_res_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_pca_raw', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [5]:
aae_bl_path = '/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20250318_replot_mosq/aae-bl/1.QC.h5ad'
aae_dv_path = '/public21/home/sc90258/huangying/analysis/12.pan_mosquito/20250224_denv_mosq/aae_denv/1.QC.h5ad'

aae_bl = sc.read(aae_bl_path)
aae_bl.obs_names = [f'{i}-bl' for i in aae_bl.obs_names]
aae_dv = sc.read(aae_dv_path)
aae_dv.obs_names = [f'{i}-dv' for i in aae_dv.obs_names]

adata = anndata.concat([aae_bl, aae_dv], join="inner", label="batch", keys=['aae-bl', 'aae-dv'])
adata

AnnData object with n_obs × n_vars = 30627 × 19269
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'mt_outlier', 'genes_outlier', 'batch'

In [22]:
adata.obs['cluster'] = ann.obs['cluster']
adata = adata[~adata.obs['cluster'].isna()]
adata.obs['cmp'] = adata.obs['batch'].map(lambda x: 1 if x == 'aae-dv' else 0)
adata

/tmp/ipykernel_100024/120733837.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cluster'] = ann.obs['cluster']
/tmp/ipykernel_100024/120733837.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cmp'] = adata.obs['batch'].map(lambda x: 1 if x == 'aae-dv' else 0)


AnnData object with n_obs × n_vars = 30527 × 19269
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'mt_outlier', 'genes_outlier', 'batch', 'cluster', 'cmp'

In [30]:
for cell in adata.obs['cluster'].unique():
    ofile = '1.input/' + cell.replace(r'/', '-') + '.h5ad'
    adata[adata.obs['cluster'] == cell].write(ofile)

In [21]:
import sys
sys.path.insert(0, '/public21/home/sc90258/huangying/tools/scrna-parameter-estimation-master/')
import memento